## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-14-06-46-26 +0000,bbc,UK economic growth slows but beats forecasts,https://www.bbc.com/news/articles/c0ml42ww740o...
1,2025-08-14-06-44-00 +0000,wsj,U.K. Economy Slows But Keeps Growing Despite T...,https://www.wsj.com/economy/u-k-economy-slows-...
2,2025-08-14-06-42-00 +0000,wsj,Aviva Sticks With Outlook After Profit Jump,https://www.wsj.com/business/earnings/aviva-st...
3,2025-08-14-06-39-45 +0000,nyt,Federal Agents Ramp Up D.C. Presence as Protes...,https://www.nytimes.com/2025/08/13/us/politics...
4,2025-08-14-06-37-28 +0000,nypost,US fugitive Nicholas Rossi accused of faking h...,https://nypost.com/2025/08/14/us-news/fugitive...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2435/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
110,trump,57
151,putin,17
38,new,15
135,ukraine,10
22,police,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
173,2025-08-13-17-57-20 +0000,nypost,Trump says he doesn’t believe he can convince ...,https://nypost.com/2025/08/13/us-news/trump-sa...,116
213,2025-08-13-15-03-41 +0000,nypost,Trump vows main goal of Putin Alaska summit is...,https://nypost.com/2025/08/13/world-news/trump...,107
197,2025-08-13-16-25-00 +0000,wsj,New York Attorney General Letitia James is sui...,https://www.wsj.com/politics/policy/new-york-a...,106
154,2025-08-13-19-04-10 +0000,nyt,Trump’s Affinity for Putin Will Be Tested at H...,https://www.nytimes.com/2025/08/13/us/politics...,106
250,2025-08-13-11-59-00 +0000,wsj,"Ahead of Trump-Putin Meeting, Zelensky Says He...",https://www.wsj.com/world/ahead-of-trump-putin...,105


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
173,116,2025-08-13-17-57-20 +0000,nypost,Trump says he doesn’t believe he can convince ...,https://nypost.com/2025/08/13/us-news/trump-sa...
271,58,2025-08-13-10-00-00 +0000,nypost,Sean Duffy opens up on spat with Elon Musk ove...,https://nypost.com/2025/08/13/us-news/sean-duf...
28,51,2025-08-14-04-11-33 +0000,nypost,Ex-police chief warns Chicago cops will get hu...,https://nypost.com/2025/08/14/us-news/ex-polic...
197,49,2025-08-13-16-25-00 +0000,wsj,New York Attorney General Letitia James is sui...,https://www.wsj.com/politics/policy/new-york-a...
201,40,2025-08-13-16-03-42 +0000,nypost,"Trump announces George Strait, Sylvester Stall...",https://nypost.com/2025/08/13/us-news/trump-an...
176,34,2025-08-13-17-51-40 +0000,nyt,Zelensky and Other European Leaders Work With ...,https://www.nytimes.com/2025/08/13/world/europ...
35,34,2025-08-14-03-16-56 +0000,nypost,"Woman who allegedly murdered NYC girlfriend, c...",https://nypost.com/2025/08/13/us-news/girlfrie...
240,32,2025-08-13-13-00-00 +0000,wsj,When companies realized they needed to diversi...,https://www.wsj.com/economy/trade/trumps-tarif...
237,29,2025-08-13-13-10-13 +0000,nypost,Trump rails against ‘unfair’ media quoting ‘fi...,https://nypost.com/2025/08/13/world-news/trump...
257,29,2025-08-13-10-47-47 +0000,nypost,DC Mayor Muriel Bowser backtracks on support f...,https://nypost.com/2025/08/13/us-news/dc-mayor...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
